In [1]:
# *** A simple demo to process the compiled dataset step by step
# this file is general to every fold of data in the cross validation
# after this file, the data for different folds is generated by using data_processing_updated_2.ipynb ***

In [ ]:
import pandas as pd
import numpy as np
import pickle
import copy
import requests
# prepare cell line data

pd.options.display.max_columns = None
# the folder to store our compiled dataset
prefix = 'D:/B/PROJECT B2_2/dataset/Muthene_dataset/' 
midfix = 'common files/'

# downloaded from: https://depmap.org/portal/download/
# CCLE: Cancer Cell Line Encyclopedia
CCLE_expression = pd.read_csv(prefix + midfix + 'CCLE_expression.csv')
CCLE_expression = CCLE_expression.rename(columns={'Unnamed: 0': 'depmap_id'})

response = requests.get('https://api.drugcomb.org/cell_lines')
drugcomb_cellline_allinfo = response.json()
drugcomb_cellline_allinfo_dict = {}
for i in drugcomb_cellline_allinfo:
    drugcomb_cellline_allinfo_dict[i['name']] = i

# number of cell line contained in drugcomb
print(len(drugcomb_cellline_allinfo_dict))

2320


In [3]:
# the other original files from different sources

# drug related domain knowledge
in_file = open(prefix + midfix + 'drugcomb_alldruginfo_dict.pickle', 'rb')
drugcomb_alldruginfo_dict = pickle.load(in_file)
in_file.close()

# synergy score data
drugcomb = pd.read_csv(prefix + midfix + 'drugcomb_synergy_score.csv')

# side effect data
twosides = pd.read_csv(prefix + midfix + 'twosides_side_effect.csv')

# drug-target interaction
drug_target = pd.read_csv(prefix + midfix + 'drug_target_interaction.csv')

# target-target interaction
target_target = pd.read_csv(prefix + midfix + 'target_target_interaction.csv')

drugcomb = drugcomb.rename(columns={'drug_row': 'drug1', 'drug_col': 'drug2'})
twosides = twosides[['drug1', 'drug2', 'Polypharmacy Side Effect',
                     'Side Effect Name', 'drug1_lower', 'drug2_lower', 'unified_name']]

# check the number of problematic samples based on different synergy score criteria
# zip：None
drugcomb['synergy_zip'] = drugcomb['synergy_zip'].astype('float')
print(drugcomb['synergy_zip'].max(), drugcomb['synergy_zip'].min(),
      drugcomb['synergy_zip'].mean(), drugcomb['synergy_zip'].median())

# hsa: None
drugcomb['synergy_hsa'] = drugcomb['synergy_hsa'].astype('float')
print(drugcomb['synergy_hsa'].max(), drugcomb['synergy_hsa'].min(),
      drugcomb['synergy_hsa'].mean(), drugcomb['synergy_hsa'].median())

# bliss: None
drugcomb['synergy_bliss'] = drugcomb['synergy_bliss'].astype('float')
print(drugcomb['synergy_bliss'].max(), drugcomb['synergy_bliss'].min(
), drugcomb['synergy_bliss'].mean(), drugcomb['synergy_bliss'].median())

# remove the 4 problematic data based on loewe
temp = drugcomb[drugcomb['synergy_loewe'] != '\\N']
temp_drugset = set.union(set(temp['drug1']), set(temp['drug2']))

drugcomb = drugcomb.drop(
    index=(drugcomb.loc[(drugcomb['synergy_loewe'] == '\\N')].index))
drugcomb['synergy_loewe'] = drugcomb['synergy_loewe'].astype('float')

print(drugcomb['synergy_loewe'].max(), drugcomb['synergy_loewe'].min(
), drugcomb['synergy_loewe'].mean(), drugcomb['synergy_loewe'].median())
# the loewe synergy score is lower than others

print('total drug number:', len(temp_drugset))

128.433573916656 -86.1689137467893 -1.1658416418422066 -0.46167060284044703
87.93870313788229 -90.13426667491999 -1.228943063125428 -1.0785883625221
207.19724904478198 -95.0502059205476 -1.3513368723624046 -0.543827700211486
82.9982087530354 -101.379902323728 -8.991664263881768 -4.3698299642188
total drug number: 213


In [4]:
drugcomb.head()

,block_id,drug1,drug2,cell_line_name,study_name,tissue_name,conc_row_unit,conc_col_unit,ic50_row,ic50_col,ri_row,ri_col,css_row,css_col,css_ri,S_sum,S_mean,S_max,synergy_zip,synergy_loewe,synergy_hsa,synergy_bliss,drug_row_clinical_phase,drug_col_clinical_phase,drug_row_target_name,drug_col_target_name,drug1_lower,drug2_lower,unified_name
0,462981,prochlorperazine,tamoxifen,TC-32,NCATS_DIPG,bone,uM,uM,0.011618,20.000000,-11.816,-7.484,48.080,2.795,25.4375,44.7375,35.0875,32.9215,12.411371,1.358891,0.880952,14.291624,4,4,Prelamin-A/C; Cytochrome P450 2D6; Cytochrome ...,Anti-estrogen binding site (AEBS); Estrogen re...,prochlorperazine,tamoxifen,prochlorperazine_tamoxifen
1,463078,prochlorperazine,mefloquine,TC-32,NCATS_DIPG,bone,uM,uM,0.003762,3.707453,0.815,2.410,2.855,-0.602,1.1265,-2.0985,-0.4860,-1.2835,-1.954036,-1.954405,-4.311522,-1.398677,4,4,Prelamin-A/C; Cytochrome P450 2D6; Cytochrome ...,\N,prochlorperazine,mefloquine,prochlorperazine_mefloquine
2,463260,prochlorperazine,Gefitinib,TC-32,NCATS_DIPG,bone,uM,uM,0.100000,20.000000,-4.949,3.439,20.059,3.567,11.8130,13.3230,12.5680,8.3740,0.085606,-1.472113,-2.211179,4.015759,4,4,Prelamin-A/C; Cytochrome P450 2D6; Cytochrome ...,Epidermal growth factor receptor erbB1; Recept...,prochlorperazine,gefitinib,prochlorperazine_gefitinib
3,463534,Dasatinib,prochlorperazine,TC-32,NCATS_DIPG,bone,uM,uM,0.023071,0.046578,13.931,-7.661,10.154,3.041,6.5975,0.3275,3.4625,-7.3335,2.964407,-3.712887,-2.208605,4.015463,4,4,Tyrosine-protein kinase LCK; Tyrosine-protein ...,Prelamin-A/C; Cytochrome P450 2D6; Cytochrome ...,dasatinib,prochlorperazine,dasatinib_prochlorperazine
4,132914,Imatinib,bleomycin,SF-268,ALMANAC,brain,uM,uM,1.997124,0.217270,-1.206,14.845,16.977,16.503,16.7400,3.1010,9.9205,1.8950,-5.967873,-10.254135,1.665557,3.507983,4,4,Tyrosine-protein kinase ABL; Platelet-derived ...,\N,imatinib,bleomycin,imatinib_bleomycin


In [5]:
twosides.head()

,drug1,drug2,Polypharmacy Side Effect,Side Effect Name,drug1_lower,drug2_lower,unified_name
0,imipramine,methotrexate,C0043096,loss of weight,imipramine,methotrexate,methotrexate_imipramine
1,imipramine,methotrexate,C0032285,neumonia,imipramine,methotrexate,methotrexate_imipramine
2,imipramine,methotrexate,C0009676,confusion,imipramine,methotrexate,methotrexate_imipramine
3,imipramine,methotrexate,C0398353,Hypoventilation,imipramine,methotrexate,methotrexate_imipramine
4,imipramine,methotrexate,C0000737,abdominal pain,imipramine,methotrexate,methotrexate_imipramine


In [6]:
# give an order to involved drugs (absolute ids)
drugset = list(set.union(set(drugcomb['drug1_lower']), set(drugcomb['drug2_lower'])))
drugset.sort()
drug2absid_dict = {drug: i for i, drug in enumerate(drugset)} # based on drugcomb

# give an order to involved targets (absolute ids)
drug_target = drug_target.rename(columns={'target': 'gene symbol'})
drug_target = drug_target[['drug', 'drug_lower', 'gene symbol']]
temp = set(drug_target['gene symbol'])

target_target = target_target[['gene1 symbol', 'gene2 symbol']]
target_target.iloc[103608]['gene1 symbol'] = 'WTIP'  # modify outdated sample
targetset = list(set.union(set(target_target['gene1 symbol']), set(target_target['gene2 symbol'])))
targetset.sort()

# unify the all the target name to lowercase
temp_2 = set([target.lower() for target in temp])
targetset_2 = set([target.lower() for target in targetset])
print(temp-set(targetset))

print(temp_2-targetset_2)

print(len(temp), len(temp_2), len(targetset), len(targetset_2))

temp_3 = pd.DataFrame([target.lower() for target in targetset])
print(temp_3.iloc[temp_3.duplicated()[temp_3.duplicated() == True].index])

target_target[(target_target['gene1 symbol'].str.lower() == 'wtip') | (
    target_target['gene2 symbol'].str.lower() == 'wtip')]

# incorporate targets in target-target interaction into the target set
targetset = list(set.union(set(targetset), temp))
targetset.sort()
target2absid_dict = {target: i for i, target in enumerate(targetset)} # based on target_target and drug_target

print(len(set(drugcomb['unified_name'])), len(set(twosides['unified_name'])))

# make the number of drug pairs in drugcomb and twosides is the same
twosides_pairset = set(twosides['unified_name'])
drugcomb_pairset = set(drugcomb['unified_name'])
inter_pairset = list(set.intersection(twosides_pairset, drugcomb_pairset))
# fix the order of drug pairs
inter_pairset.sort()

drugcomb_reduced = []  # reduce the extra drug pairs in drugcomb
for row in np.array(drugcomb):
    if(row[-1] in inter_pairset):
        drugcomb_reduced.append(row)

drugcomb = pd.DataFrame(drugcomb_reduced, columns=drugcomb.columns)
print(len(set(drugcomb['unified_name'])), len(set(twosides['unified_name'])))

# currently both datasets have 2764 drug pairs and this number will be further shortened due to the limited acquisition of cell line information

{'CA13', 'ALPPL2', 'ACCN1', 'ABP1', 'ACCN2', 'GPR19'}
{'ca13', 'abp1', 'gpr19', 'accn1', 'alppl2', 'accn2'}
825 825 12796 12796
Empty DataFrame
Columns: [0]
Index: []
2837 2764
2764 2764


In [7]:
# start to determine the number of cell line according to existing cell line information
# 'ACH-'' is the id of depmap, which is a cell line dataset

In [8]:
drugcomb_cellline_allinfo_dict['DU-145']

{'name': 'DU-145',
 'synonyms': 'DU-145; DU-145; Du-145; DU 145; DU_145',
 'cellosaurus_accession': 'CVCL_0105',
 'disease_id': 'C4863',
 'id': 16,
 'tissue_id': 9,
 'depmap_id': 'ACH-000979',
 'cell_model_passport_id': 'SIDM00120',
 'cosmic_id': 905935,
 'ccle_name': 'DU145_PROSTATE',
 'expression_data': 'CellModelPassport'}

In [9]:
cellline_set = list(set(drugcomb['cell_line_name']))
cellline_set.sort()
cellline2absid_dict = {cellline: i for i, cellline in enumerate(cellline_set)}  # complete cell line set, based on drugcomb

counter = 0
# contain basic information of all involved cell lines
drugcomb_cellline_info_table = []
vacant_cellline = []  # cell lines that are not in drugcomb dataset
# cell lines that are in drugcomb dataset originated from depmap
existing_cellline_depmap = []
# cell lines that are in drugcomb dataset originated from CMP
existing_cellline_CMP = []
existing_cellline = []
for cellline in cellline_set:
    if drugcomb_cellline_allinfo_dict[cellline]['expression_data'] == 'NA':
        vacant_cellline.append(cellline)
        counter += 1
    elif drugcomb_cellline_allinfo_dict[cellline]['expression_data'] == 'BROAD_DepMap':
        existing_cellline_depmap.append(cellline)
    elif drugcomb_cellline_allinfo_dict[cellline]['expression_data'] == 'CellModelPassport':
        existing_cellline_CMP.append(cellline)
    else:
        existing_cellline.append(cellline)  # other cases

    drugcomb_cellline_info_table.append(
        drugcomb_cellline_allinfo_dict[cellline])

print(vacant_cellline)  # KBM-7 cannot be found
print('vacant cellline number:', counter)

drugcomb_cellline_info_table = pd.DataFrame(drugcomb_cellline_info_table)

# test for depmap based cell line data, all of them can be the found in CCLE_expression file
expression_in_BROAD_DepMap = drugcomb_cellline_info_table[
    drugcomb_cellline_info_table['expression_data'] == 'BROAD_DepMap']

temp1 = set(CCLE_expression['depmap_id'])
temp2 = set(expression_in_BROAD_DepMap['depmap_id'])
print(temp2-temp1)

drugcomb_cellline_info_table.head() 
# include all information retrieved from drugcomb 
# but not all of cell lines can find expression data neither in CMP nor depmap

['3D7', '501-mel', 'CBRC002', 'CBRC007', 'CBRC008', 'CBRC013', 'CBRC014', 'CBRC016', 'CBRC017', 'CBRC026', 'CBRC029', 'CBRC034', 'CBRC042', 'CBRC043', 'CBRC050', 'CBRC056', 'CBRC058', 'COLO320DM', 'DD2', 'DIPG25', 'EFM192B', 'HB3', 'HL-60(TB)', 'KBM-7', 'LNCAP', 'MCF7F10016', 'MCF7F1003', 'MCF7T52', 'MDA-MB-435', 'MDAMB134V1', 'NCI-H322M', 'NCI/ADR-RES', 'SNB-19', 'TC-32', 'UWB1289+BRCA1', 'WM1716', 'WM1745', 'WM3526', 'WM3619', 'WT2-iPS']
vacant cellline number: 40
set()


,name,synonyms,cellosaurus_accession,disease_id,id,tissue_id,depmap_id,cell_model_passport_id,cosmic_id,ccle_name,expression_data
0,3D7,3D7,NA,C34797,2288,3,NA,NA,NaN,NA,NA
1,451Lu,451Lu; 451LU; 451-LU; WM 451-Lu; WM-451Lu; WM-...,CVCL_6357,C3510,129,10,ACH-001002,SIDM01240,1287706.0,451LU_SKIN,CellModelPassport
2,501-mel,501-mel; 501-MEL; 501 mel; 501 Mel; 501Mel; 50...,CVCL_4633,C3224,2258,10,NA,NA,NaN,NA,NA
3,786-0,786-0; 786O; 786-0; 786.O; 786-O RCC; RCC 786-...,CVCL_1051,C9385,1,5,ACH-000649,SIDM00125,905947.0,786O_KIDNEY,BROAD_DepMap
4,A101D,A101D; A-101D,CVCL_1057,C3224,150,10,ACH-000008,SIDM00801,910921.0,A101D_SKIN,BROAD_DepMap


In [11]:
# existing_cellline_depmap: cell line that can find expression data in CCLE dataset
# consider CCLE dataset at first becuase it has more data
drugcomb_depmap = drugcomb[drugcomb['cell_line_name'].isin(existing_cellline_depmap)]

# remaining cell lines
print('remaining cell lines after using CCLE:', len(set(drugcomb_depmap['cell_line_name'])))
# remaining drugs
print('remaining drugs after using CCLE:', len(set(drugcomb_depmap['drug1_lower']).union(set(drugcomb_depmap['drug2_lower']))))

# store the number of drug-drug pairs that current each cell line corresponds to
cellline_num_list = []
for cellline in set(drugcomb_depmap['cell_line_name']):
    num = drugcomb_depmap[drugcomb_depmap['cell_line_name'] == cellline].shape[0]
    cellline_num_list.append([cellline, num])

cellline_num_list = pd.DataFrame(cellline_num_list)
cellline_num_list = cellline_num_list.sort_values([1], ascending=(False)).reset_index(drop=True)

# [0]: total drug-drug-cell line samples according to remaining cell lines
# [1]: total drug-drug-cell line samples according to further selected cell lines of remaining cell lines
# every top-40 cell line corresponding to at least 200 drug-drug pairs, top-60: 37
selected_cellline_num = 60
print('before further selecting cell lines:', drugcomb_depmap.shape[0], 'after:', cellline_num_list[:selected_cellline_num][1].sum())

# based on selected_cellline_num
cellline_set_reduced = list(cellline_num_list[:selected_cellline_num][0])
cellline_set_reduced.sort()
cellline2relid_dict = {cellline: i for i, cellline in enumerate(cellline_set_reduced)}

# further reduce the cell line number in depmap based dataset
drugcomb_depmap_reduced = drugcomb_depmap[drugcomb_depmap['cell_line_name'].isin(cellline2relid_dict.keys())].reset_index(drop=True)
drugset_reduced = list(set(drugcomb_depmap_reduced['drug1_lower']).union(set(drugcomb_depmap_reduced['drug2_lower'])))
drugset_reduced.sort()
drug2relid_dict = {drug: i for i, drug in enumerate(drugset_reduced)}  # based on selected_cellline_num

# current drug, target, cellline set have the complete number of samples
print('absolute entity number:', len(drug2absid_dict), len(target2absid_dict), len(cellline2absid_dict))
# currently only determine the relative drug and cell line ids (not including target ids)
print('relative entity number:', len(drug2relid_dict), len(cellline2relid_dict))

print('the least number of drugs corresponding to one cell line:', cellline_num_list.iloc[selected_cellline_num-1][1])

# under selected CCLE cell lines:
# interestingly, top40 cell lines only include 48 drugs, top40-60 cell lines include at least 37 drugs
# which means that, drugs corresponding to different cell lines could be totally different
# adding top40-top60 could increase the difficulty of model prediction

# *** the following experiments are based on drugcomb_cellline_depmap_reduced dataset and drug2relid_dict ***

remaining cell lines after using CCLE: 93
remaining drugs after using CCLE: 106
before further selecting cell lines: 11907 after: 11166
absolute entity number: 213 12802 147
relative entity number: 106 60
the least number of drugs corresponding to one cell line: 37


In [13]:
print(drugcomb_depmap_reduced.shape) # samples after selecting fixed number of cell lines
drugcomb_depmap_reduced.head()

(11166, 29)


,block_id,drug1,drug2,cell_line_name,study_name,tissue_name,conc_row_unit,conc_col_unit,ic50_row,ic50_col,ri_row,ri_col,css_row,css_col,css_ri,S_sum,S_mean,S_max,synergy_zip,synergy_loewe,synergy_hsa,synergy_bliss,drug_row_clinical_phase,drug_col_clinical_phase,drug_row_target_name,drug_col_target_name,drug1_lower,drug2_lower,unified_name
0,132914,Imatinib,bleomycin,SF-268,ALMANAC,brain,uM,uM,1.997124,0.217270,-1.206,14.845,16.977,16.503,16.7400,3.1010,9.9205,1.8950,-5.967873,-10.254135,1.665557,3.507983,4,4,Tyrosine-protein kinase ABL; Platelet-derived ...,\N,imatinib,bleomycin,imatinib_bleomycin
1,135848,paclitaxel,allopurinol,SF-268,ALMANAC,brain,uM,uM,0.026675,4.675736,29.318,0.146,17.788,4.071,10.9295,-18.5345,-3.8025,-18.3885,-5.326121,-22.472343,-9.719076,-9.262139,4,4,Tubulin; Integrin alpha-V/beta-3; Prelamin-A/C...,Xanthine dehydrogenase; Ras-related protein Ra...,paclitaxel,allopurinol,paclitaxel_allopurinol
2,135980,Fulvestrant,docetaxel,SF-268,ALMANAC,brain,uM,uM,4.000000,0.030177,-14.037,4.741,3.049,17.096,10.0725,19.3685,14.7205,5.3315,2.377254,-6.237605,-4.144442,9.601456,4,4,Estrogen receptor alpha; Estrogen receptor bet...,\N,fulvestrant,docetaxel,fulvestrant_docetaxel
3,137713,letrozole,cyclophosphamide,SF-268,ALMANAC,brain,uM,uM,0.044412,33.509424,-1.077,-0.379,2.678,1.556,2.1170,3.5730,2.8450,2.4960,0.316095,1.835797,2.416700,4.009233,4,4,Cytochrome P450 19A1,\N,letrozole,cyclophosphamide,letrozole_cyclophosphamide
4,138370,Imatinib,hydroxyurea,SF-268,ALMANAC,brain,uM,uM,0.542648,1.398216,1.269,0.129,-1.341,-0.498,-0.9195,-2.3175,-1.6185,-2.1885,0.036013,-0.616815,-3.276514,-1.881524,4,4,Tyrosine-protein kinase ABL; Platelet-derived ...,Ribonucleoside-diphosphate reductase RR1,imatinib,hydroxyurea,imatinib_hydroxyurea


In [14]:
# retrieve the expression data for selected top cell lines like Transynergy method

# 1. determine the number of targets interact with drugs in drugset
drug_target_depmap_reduced = drug_target[drug_target['drug_lower'].isin(list(drug2relid_dict.keys()))].reset_index(drop=True)

print('len(drug2relid_dict.keys():', len(drug2relid_dict.keys()))
# the genes on which its expression data is calculated for certain cell line
gene4cellline = set(drug_target_depmap_reduced['gene symbol'])

CCLE_gene_name = CCLE_expression.columns
CCLE_gene = set([gene[:gene.find('(')-1] for gene in CCLE_gene_name[1:]])
# CCLE column name to gene name
CCLE_gene_mapping = {gene[:gene.find('(')-1]: gene for gene in CCLE_gene_name[1:]}

for i in gene4cellline - CCLE_gene:
    gene4cellline.remove(i)
# doublecheck, should be an empty set, ensure all expression data based on genes can be found in CCLE
print(gene4cellline - CCLE_gene)

print(CCLE_expression.iloc[0])

# obtain expression data on selected genes
CCLE_expression_depmap_reduced = pd.DataFrame(CCLE_expression, columns=[
                                              'depmap_id']+[CCLE_gene_mapping[genename] for genename in gene4cellline])

cellline_name2depmap = {}  # based on cellline2relid_dict
for name in cellline2relid_dict.keys():
    # name: cell line name
    value = np.array(drugcomb_cellline_info_table[drugcomb_cellline_info_table['name'] == name]['depmap_id'])[0]
    # value: depmap id
    cellline_name2depmap[name] = value

temp_list1 = []
temp_list2 = []
temp_set = set()
for name in cellline2relid_dict:  # based on the order of cellline2relid_dict
    temp = CCLE_expression_depmap_reduced[CCLE_expression_depmap_reduced['depmap_id'] == cellline_name2depmap[name]]
    temp_list1.append(temp)  # expression data for one cell line
    temp_list2.append(name)  # name for one cell line
    if temp.shape[0] == 0:  # data that cannot be found
        temp_set.add(cellline_name2depmap[name])
        print(cellline_name2depmap[name])
        print(name)

assert len(temp_list1) == len(temp_list2), 'len(temp_list1) != len(temp_list2)'
# overwrite previous CCLE_expression_depmap_reduced
CCLE_expression_depmap_reduced = pd.concat(temp_list1)
CCLE_expression_depmap_reduced.insert(0, 'cellline_name', temp_list2, allow_duplicates=False)

print('CCLE_expression_depmap_reduced.shape:', CCLE_expression_depmap_reduced.shape)
CCLE_expression_depmap_reduced.head()

len(drug2relid_dict.keys(): 106
set()
depmap_id                  ACH-001113
TSPAN6 (7105)                  4.9905
TNMD (64102)                        0
DPM1 (8813)                    7.2737
SCYL3 (57147)                 2.76553
                              ...    
POLR2J3 (548644)              5.78188
H2BE1 (114483833)                   0
AL445238.1 (647264)                 0
GET1-SH3BGR (106865373)      0.799087
AC113348.1 (102724657)              0
Name: 0, Length: 19178, dtype: object
CCLE_expression_depmap_reduced.shape: (60, 679)


,cellline_name,depmap_id,CHRNA4 (1137),CHRNA2 (1135),FABP7 (2173),TGFBR1 (7046),SCN10A (6336),MMP14 (4323),PSMD1 (5707),CA12 (771),TBXAS1 (6916),PTGER4 (5734),CHRNA6 (8973),HIPK3 (10114),SLC47A1 (55244),DCK (1633),KIT (3815),DHRS1 (115817),DHRS7C (201140),SCN8A (6334),OGG1 (4968),HPGD (3248),CHRNE (1145),ZAP70 (7535),GPR27 (2850),FLT1 (2321),KCNA1 (3736),DHODH (1723),KCNQ1 (3784),GNRHR (2798),FGR (2268),PDE11A (50940),GPR12 (2835),SCN11A (11280),TSSK2 (23617),CA2 (760),MAP3K11 (4296),HTR6 (3362),GPR84 (53831),GPR151 (134391),RDH10 (157506),RDH8 (50700),TAAR6 (319100),CHRNA1 (1134),SSTR2 (6752),AR (367),MAP4K4 (9448),GAK (2580),FDPS (2224),SCAP (22937),FABP4 (2167),FLT3 (2322),NOS2 (4843),SYK (6850),OPRM1 (4988),GPR142 (350383),SBK1 (388228),HRH2 (3274),UTS2R (2837),LYN (4067),PBK (55872),GPR6 (2830),EPHA3 (2042),PLD1 (5337),FGFR4 (2264),LTB4R (1241),AURKA (6790),RBP5 (83758),CACNA1G (8913),JAK3 (3718),PASK (23178),CYP2C9 (1559),SLC6A20 (54716),ERBB2 (2064),NQO2 (4835),NMUR2 (56923),THRB (7068),GPR85 (54329),SLC22A2 (6582),MC4R (4160),CYP2B6 (1555),RIPK1 (8737),CA13 (377677),ANXA1 (301),MAP3K21 (84451),NR3C2 (4306),NPFFR2 (10886),HMGCR (3156),MC2R (4158),MAP3K9 (4293),GPR61 (83873),RDH12 (145226),TAAR1 (134864),NR3C1 (2908),HSD11B1 (3290),LPAR3 (23566),STYK1 (55359),PDE10A (10846),EPX (8288),SIK3 (23387),GPR37L1 (9283),HDAC2 (3066),AOX1 (316),PDE1A (5136),NR2C2 (7182),IRAK4 (51135),GPR21 (2844),HTR3C (170572),MLNR (2862),GALR2 (8811),POLA2 (23649),TLR7 (51284),RGR (5995),MKNK1 (8569),FGFRL1 (53834),CDH5 (1003),NMUR1 (10316),CA5B (11238),NR2E1 (7101),CA3 (761),CCNC (892),LIMK2 (3985),CHRFAM7A (89832),FABP6 (2172),TRIB3 (57761),NTRK2 (4915),CHRNG (1146),CA7 (766),NR1I2 (8856),HTR2C (3358),FYN (2534),CA4 (762),EPHB1 (2047),TLR8 (51311),CDK19 (23097),ADRA1B (147),MELK (9833),FABP3 (2170),KCNMA1 (3778),SLC5A7 (60482),HSD17B14 (51171),TIE1 (7075),TUBB2A (7280),NIM1K (167359),SLC6A7 (6534),TUBD1 (51174),PDPK1 (5170),PHKG1 (5260),LPAR1 (1902),PDE4C (5143),MAP2K5 (5607),DHRSX (207063),HIPK4 (147746),NR1H2 (7376),TUBA1C (84790),RRM1 (6240),PIN1 (5300),CA9 (768),MAPK15 (225689),CCR1 (1230),AGTR2 (186),FABP1 (2168),DECR1 (1666),RIPK3 (11035),ANPEP (290),NPY1R (4886),PSMB5 (5693),GPR52 (9293),GPR148 (344561),ATIC (471),FGFR3 (2261),POLA1 (5422),TBXA2R (6915),CA6 (765),SERPINA7 (6906),ADORA1 (134),MUSK (4593),EPHB4 (2050),GC (2638),HTR2B (3357),MAPK11 (5600),SLC6A3 (6531),RARG (5916),RXFP3 (51289),DECR2 (26063),CYP3A4 (1576),HTR3E (285242),ADRA1A (148),CSNK1E (1454),GPR19 (2842),MATK (4145),NTSR2 (23620),SLC6A4 (6532),MAP3K20 (51776),STK10 (6793),SIK2 (23235),RARA (5914),NPC1L1 (29881),TRIB2 (28951),PDE2A (5138),KCNH2 (3757),SSTR3 (6753),OPRD1 (4985),FABP2 (2169),DRD1 (1812),FABP9 (646480),MCHR1 (2847),LRRK2 (120892),RDH14 (57665),ACVR2A (92),ADRB2 (154),SLC12A3 (6559),PLK4 (10733),CCR5 (1234),TNF (7124),BTK (695),APLNR (187),DRD5 (1816),HTR1A (3350),ADRA1D (146),PTGER2 (5732),EPHA2 (1969),CHRNB1 (1140),CMPK1 (51727),CCR2 (729230),IDO1 (3620),EPHA5 (2044),EGFR (1956),GPR83 (10888),DHRS4L2 (317749),CTNNB1 (1499),NR6A1 (2649),DHRS2 (10202),CHRND (1144),IRAK1 (3654),AURKB (9212),RDH11 (51109),TRHR (7201),PDE5A (8654),EPHA8 (2046),GPR37 (2861),UPP2 (151531),MERTK (10461),PGR (5241),DHRS3 (9249),NEK11 (79858),PXDNL (137902),CRABP1 (1381),BRS3 (680),PTK6 (5753),RBP7 (116362),MMP8 (4317),NFKB1 (4790),TUBB2B (347733),TXK (7294),ESRRB (2103),PDE4B (5142),HPRT1 (3251),BRAF (673),EPHA4 (2043),PIM1 (5292),LPO (4025),TUBA3C (7278),STAT5B (6777),HUNK (30811),SLCO2B1 (11309),HCK (3055),PMP2 (5375),SRMS (6725),GPR3 (2827),SLC6A16 (28968),THRA (7067),CSK (1445),TESK1 (7016),PDSS1 (23590),TSSK6 (83983),STK35 (140901),FGFR2 (2263),PTGS1 (5742),PDSS2 (57107),PKDCC (91461),PDGFRL (5157),RRH (10692),ABL2 (27),QRFPR (84109),SCN2A (6326),TPO (7173),MC3R (4159),CBR1 (873),TYMS (7298),TEC (7006),NPY4R (5540),CSNK1A1 (1452),BDH2 (56898),AGTR1 (185),CYP2C19 (1557),TNNI3K (51086),GPR22 (2845),SLC6A19 (340024),PDE6B (5158),ADORA3 (140),CXCR4 (7852),MYT1 (4661),TUBA1B (10

In [60]:
print(drug_target_depmap_reduced.shape, len(set(drug_target_depmap_reduced['gene symbol'])))
# the first output is the DTI set, including 2332 DTIs, while only 679 targets/genes are included in these 2332 DTIs.

(2332, 3) 679


In [ ]:
# the order of CCLE_expression_depmap_reduced is based on cellline2relid_dict

In [15]:
# after screening samples based on existing CCLE, current determined datasets include:
print('drugcomb_depmap_reduced.shape:', drugcomb_depmap_reduced.shape)
print('drug_target_depmap_reduced.shape:', drug_target_depmap_reduced.shape)

print('len(drug2absid_dict):', len(drug2absid_dict))
print('len(target2absid_dict):', len(target2absid_dict))
print('len(cellline2absid_dict):', len(cellline2absid_dict))

print('len(drug2relid_dict):', len(drug2relid_dict))
print('len(cellline2relid_dict):', len(cellline2relid_dict))

# next we need to determine the twosides and target sets according to the current determined datasets again

drugcomb_depmap_reduced.shape: (11166, 29)
drug_target_depmap_reduced.shape: (2332, 3)
len(drug2absid_dict): 213
len(target2absid_dict): 12802
len(cellline2absid_dict): 147
len(drug2relid_dict): 106
len(cellline2relid_dict): 60


In [ ]:
# a chessboard of rel_drug*rel_drug*cell line number size
# due to the reduction of drug/drug-drug pairs, the adverse effect samples also decrease

In [18]:
# start to determine the twosides dataset
twosides_pairset = set(twosides['unified_name'])
drugcomb_depmap_reduced_pairset = set(drugcomb_depmap_reduced['unified_name'])
inter_depmap_reduced_pairset = list(set.intersection(twosides_pairset, drugcomb_depmap_reduced_pairset))
inter_depmap_reduced_pairset.sort()

twosides_depmap_reduced = []
for row in np.array(twosides):
    if(row[-1] in inter_depmap_reduced_pairset):
        twosides_depmap_reduced.append(row)

twosides_depmap_reduced = pd.DataFrame(twosides_depmap_reduced, columns=twosides.columns)
print('drug pair numbers in drugcomb and towsides:', len(set(drugcomb_depmap_reduced['unified_name'])), len(set(twosides_depmap_reduced['unified_name'])))
print('twosides_depmap_reduced.shape:', twosides_depmap_reduced.shape)

# after processing, 346 drug pairs are remained in drugcomb_depmap and twosides_depmap, while in twosides_depmap, 2446 samples are remained
# before processing: 2764 drug pairs are remained
# after processing, drug-drug-cell line pairs focus on fewer drug/drug-drug pairs
# although the total number of drug-drug-cell line pairs increase

# start to determine the target-target dataset
# based on our current selected meta-path types (DTD, DTTD, DTTTD),
# at least one target in target_target samples should be in the target set of drug_target_depmap_reduced
targetset4drug_target_depmap_reduced = set(drug_target_depmap_reduced['gene symbol'])

target_target_depmap_reduced = []
for row in np.array(target_target): 
    if (row[0] in targetset4drug_target_depmap_reduced) or (row[1] in targetset4drug_target_depmap_reduced):
        target_target_depmap_reduced.append(row)
target_target_depmap_reduced = pd.DataFrame(target_target_depmap_reduced, columns=target_target.columns)

targetset_reduced = set(target_target_depmap_reduced['gene1 symbol']).union(target_target_depmap_reduced['gene2 symbol'])
targetset_reduced = list(targetset_reduced.union(targetset4drug_target_depmap_reduced))
targetset_reduced.sort()
target2relid_dict = {target: i for i, target in enumerate(targetset_reduced)}

print('target_target_depmap_reduced.shape:', target_target_depmap_reduced.shape)
print('len(target2relid_dict):', len(target2relid_dict))
print('len(targetset_reduced):', len(targetset_reduced))

drug pair numbers in drugcomb and towsides: 346 346
twosides_depmap_reduced.shape: (2446, 7)
target_target_depmap_reduced.shape: (91785, 2)
len(target2relid_dict): 12217
len(targetset_reduced): 12217


In [23]:
# previous: 7089 drug-drug-cell lines, 18832 AE samples
# now: 11166 drug-drug-cell lines, 2446 AE samples

# previous: 2694 drug pairs, 159 drugs
# now: 346 drug pairs, 106 drugs

# each kind of drug pair corresponds to similar number of AE labels (previous and now), but the total drug pair varieties decrease dramatically
print(18832/2694, 2446/346) # AE

# although the number of AE that each drug pair corresponds to is similar, but for now each drug pair corresponds much more cell lines 
print(7089/2694, 11166/346) # TE

# previous vs now: for each drug pair, its averaging AE label number: unchanged, its averaging TE number: increasing a lot (3 to 30)

print('total drug, target, cell line numbers:', len(drug2relid_dict), len(target2relid_dict), len(cellline2relid_dict))

6.990348923533778 7.069364161849711
2.631403118040089 32.27167630057804
total drug, target, cell line numbers: 106 12217 60


In [24]:
# generate drug-drug adjacent matrices based on relative ids and drug lowercase names
# current do not double the drug-drug pairs by reversing A-B to B-A (thus current adjacent matices are not symmetric)
# to avoid data leakage, only to double drug-drug pairs in the specific training set

duplicated_te = []
drug_te_drug_matrix = np.zeros((len(drug2relid_dict), len(drug2relid_dict)))
for _, row in drugcomb_depmap_reduced.iterrows():
    # row
    temp1 = drug2relid_dict[row['drug1_lower']]
    # col
    temp2 = drug2relid_dict[row['drug2_lower']]
    # check whether exists A/B or B/A cases
    if(drug_te_drug_matrix[temp2, temp1] == 1):
        duplicated_te.append(row)
    drug_te_drug_matrix[temp1, temp2] = 1

duplicated_se = []
drug_se_drug_matrix = np.zeros((len(drug2relid_dict), len(drug2relid_dict)))
for _, row in twosides_depmap_reduced.iterrows():
    # row
    temp1 = drug2relid_dict[row['drug1_lower']]
    # col
    temp2 = drug2relid_dict[row['drug2_lower']]
    # check whether exists A/B or B/A cases
    if(drug_se_drug_matrix[temp2, temp1] == 1):
        duplicated_se.append(row)
    drug_se_drug_matrix[temp1, temp2] = 1

print(drug_se_drug_matrix.sum(), drug_te_drug_matrix.sum())
# *** no duplicated A-B/B-A drug pairs if the two lists are empty ***
# *** (every sample in current drugcomb and twosides does not have the reverse counterpart) ***
print(duplicated_te, duplicated_se)

# *** re-generate the drug_te_drug_matrix according to pre-defined threshold for synergy score values ***

# A p-value, or probability value, is a number describing how likely it is that your data would have occurred by random chance (i.e. that the null hypothesis is true).
# A p-value less than 0.05 (typically ≤ 0.05) is statistically significant. It indicates strong evidence against the null hypothesis, as there is less than a 5% probability the null is correct (and the results are random). Therefore, we reject the null hypothesis, and accept the alternative hypothesis.

# PPF of the standard normal distribution for the probability 1-a = 0.95
# scipy.stats.norm.ppf(0.95)=1.64, https://educationalresearchtechniques.com/2018/09/24/z-scores-and-inferential-stats-in-python/
# do not need to do 0-1 normalization
# based on loewe synergy score

duplicated_te = []
synergy_score_qualified = []
drug_te_drug_matrix = np.zeros((len(drug2relid_dict), len(drug2relid_dict)))
used_synergy_score = 'synergy_loewe'
mean = np.mean(drugcomb_depmap_reduced[used_synergy_score])
std = np.std(drugcomb_depmap_reduced[used_synergy_score])
for _, row in drugcomb_depmap_reduced.iterrows():
    # row
    temp1 = drug2relid_dict[row['drug1_lower']]
    # col
    temp2 = drug2relid_dict[row['drug2_lower']]
    z_score = (float(row[used_synergy_score]) - mean) / std # z-score for current sample
    # check A-B/B-A
    if(drug_te_drug_matrix[temp2, temp1] == 1):
        duplicated_te.append(row)
    if z_score >= 1.64:
        drug_te_drug_matrix[temp1, temp2] = 1
        synergy_score_qualified.append(float(row[used_synergy_score]))
        
print(duplicated_te)
# threshold: 17.06, synergy_score_qualified: total number of synergy score that meets the pre-defined threshold
# based on each drug-drug pair, maybe there are many drug-drug-cell line samples have a synergy score larger than pre-defined threshold
print(len(synergy_score_qualified), np.max(synergy_score_qualified),
      np.min(synergy_score_qualified), np.mean(synergy_score_qualified))
# 36/346 of drug-drug pairs have the te edge in drug_te_drug_matrix
print(drug_te_drug_matrix.sum())

346.0 346.0
[] []
[]
84 70.1360399671155 17.0599025704251 24.690158328988062
37.0


In [25]:
# after generating asymmetric drug-drug matrices, generate target-target (symmetric) and drug-target (asymmetric) matrices
target_target_matrix = np.zeros((len(target2relid_dict), len(target2relid_dict)))
for row in np.array(target_target_depmap_reduced):
    # row
    temp1 = target2relid_dict[row[0]]
    # col
    temp2 = target2relid_dict[row[1]]
    target_target_matrix[temp1, temp2] = 1
    target_target_matrix[temp2, temp1] = 1

drug_target_matrix = np.zeros((len(drug2relid_dict), len(target2relid_dict)))
for row in np.array(drug_target_depmap_reduced):
    # drug
    temp1 = drug2relid_dict[row[1]]
    # target
    temp2 = target2relid_dict[row[2]]
    drug_target_matrix[temp1, temp2] = 1

In [26]:
# four main matrices to be used:
print('drug_te_drug_matrix.shape:', drug_te_drug_matrix.shape)
print('drug_se_drug_matrix.shape:', drug_se_drug_matrix.shape)
print('target_target_matrix.shape:', target_target_matrix.shape)
print('drug_target_matrix.shape:', drug_target_matrix.shape)

drug_te_drug_matrix.shape: (106, 106)
drug_se_drug_matrix.shape: (106, 106)
target_target_matrix.shape: (12217, 12217)
drug_target_matrix.shape: (106, 12217)


In [44]:
from scipy import sparse

prefix = 'D:/B/PROJECT B2_2/dataset/Muthene_dataset/'
midfix = 'common files/'

# transform the above generated dense matrices to sparse ones
# the ids in these matrics are based on drug2rel_dict and target2rel_dict

drug_se_drug_coomatrix = sparse.coo_matrix(drug_se_drug_matrix)
drug_te_drug_coomatrix = sparse.coo_matrix(drug_te_drug_matrix)
drug_target_coomatrix = sparse.coo_matrix(drug_target_matrix)
target_target_coomatrix = sparse.coo_matrix(target_target_matrix)

# store the files generated above to the specified folder (currently same to the original data folder)
sparse.save_npz(prefix + midfix + 'drug_se_drug_coomatrix.npz', 
                drug_se_drug_coomatrix) # asymmetric
sparse.save_npz(prefix + midfix + 'drug_te_drug_coomatrix.npz',
                drug_te_drug_coomatrix) # asymmetric
sparse.save_npz(prefix + midfix + 'drug_target_coomatrix.npz',
                drug_target_coomatrix) # asymmetric
sparse.save_npz(prefix + midfix + 'target_target_coomatrix.npz',
                target_target_coomatrix) # symmetric

# drug and target files before processing based on depmap reduced (but after the correction of false/redundant samples)
# based on absid_dicts
drugcomb.to_csv(prefix + midfix + 'drugcomb.csv', index=False)
twosides.to_csv(prefix + midfix + 'twosides.csv', index=False)
drug_target.to_csv(prefix + midfix + 'drug_target.csv', index=False)
target_target.to_csv(prefix + midfix + 'target_target.csv', index=False)

# drug and target files after processing based on depmap reduced
# based on relid_dicts
drugcomb_depmap_reduced.to_csv(prefix + midfix + 'drugcomb_depmap.csv', index=False)
twosides_depmap_reduced.to_csv(prefix + midfix + 'twosides_depmap.csv', index=False)
drug_target_depmap_reduced.to_csv(prefix + midfix + 'drug_target_depmap.csv', index=False)
target_target_depmap_reduced.to_csv(prefix + midfix + 'target_target_depmap.csv', index=False)

# six mapping dict generated in this file
with open(prefix + midfix + 'drug2relid_dict.pickle', 'wb') as out_file:
    pickle.dump(drug2relid_dict, out_file)
with open(prefix + midfix + 'target2relid_dict.pickle', 'wb') as out_file:
    pickle.dump(target2relid_dict, out_file)
with open(prefix + midfix + 'cellline2relid_dict.pickle', 'wb') as out_file:
    pickle.dump(cellline2relid_dict, out_file)
with open(prefix + midfix + 'drug2absid_dict.pickle', 'wb') as out_file:
    pickle.dump(drug2absid_dict, out_file)
with open(prefix + midfix + 'target2absid_dict.pickle', 'wb') as out_file:
    pickle.dump(target2absid_dict, out_file)
with open(prefix + midfix + 'cellline2absid_dict.pickle', 'wb') as out_file:
    pickle.dump(cellline2absid_dict, out_file)

In [28]:
print(CCLE_expression_depmap_reduced.shape)
CCLE_expression_depmap_reduced.head()

(60, 679)


,cellline_name,depmap_id,CHRNA4 (1137),CHRNA2 (1135),FABP7 (2173),TGFBR1 (7046),SCN10A (6336),MMP14 (4323),PSMD1 (5707),CA12 (771),TBXAS1 (6916),PTGER4 (5734),CHRNA6 (8973),HIPK3 (10114),SLC47A1 (55244),DCK (1633),KIT (3815),DHRS1 (115817),DHRS7C (201140),SCN8A (6334),OGG1 (4968),HPGD (3248),CHRNE (1145),ZAP70 (7535),GPR27 (2850),FLT1 (2321),KCNA1 (3736),DHODH (1723),KCNQ1 (3784),GNRHR (2798),FGR (2268),PDE11A (50940),GPR12 (2835),SCN11A (11280),TSSK2 (23617),CA2 (760),MAP3K11 (4296),HTR6 (3362),GPR84 (53831),GPR151 (134391),RDH10 (157506),RDH8 (50700),TAAR6 (319100),CHRNA1 (1134),SSTR2 (6752),AR (367),MAP4K4 (9448),GAK (2580),FDPS (2224),SCAP (22937),FABP4 (2167),FLT3 (2322),NOS2 (4843),SYK (6850),OPRM1 (4988),GPR142 (350383),SBK1 (388228),HRH2 (3274),UTS2R (2837),LYN (4067),PBK (55872),GPR6 (2830),EPHA3 (2042),PLD1 (5337),FGFR4 (2264),LTB4R (1241),AURKA (6790),RBP5 (83758),CACNA1G (8913),JAK3 (3718),PASK (23178),CYP2C9 (1559),SLC6A20 (54716),ERBB2 (2064),NQO2 (4835),NMUR2 (56923),THRB (7068),GPR85 (54329),SLC22A2 (6582),MC4R (4160),CYP2B6 (1555),RIPK1 (8737),CA13 (377677),ANXA1 (301),MAP3K21 (84451),NR3C2 (4306),NPFFR2 (10886),HMGCR (3156),MC2R (4158),MAP3K9 (4293),GPR61 (83873),RDH12 (145226),TAAR1 (134864),NR3C1 (2908),HSD11B1 (3290),LPAR3 (23566),STYK1 (55359),PDE10A (10846),EPX (8288),SIK3 (23387),GPR37L1 (9283),HDAC2 (3066),AOX1 (316),PDE1A (5136),NR2C2 (7182),IRAK4 (51135),GPR21 (2844),HTR3C (170572),MLNR (2862),GALR2 (8811),POLA2 (23649),TLR7 (51284),RGR (5995),MKNK1 (8569),FGFRL1 (53834),CDH5 (1003),NMUR1 (10316),CA5B (11238),NR2E1 (7101),CA3 (761),CCNC (892),LIMK2 (3985),CHRFAM7A (89832),FABP6 (2172),TRIB3 (57761),NTRK2 (4915),CHRNG (1146),CA7 (766),NR1I2 (8856),HTR2C (3358),FYN (2534),CA4 (762),EPHB1 (2047),TLR8 (51311),CDK19 (23097),ADRA1B (147),MELK (9833),FABP3 (2170),KCNMA1 (3778),SLC5A7 (60482),HSD17B14 (51171),TIE1 (7075),TUBB2A (7280),NIM1K (167359),SLC6A7 (6534),TUBD1 (51174),PDPK1 (5170),PHKG1 (5260),LPAR1 (1902),PDE4C (5143),MAP2K5 (5607),DHRSX (207063),HIPK4 (147746),NR1H2 (7376),TUBA1C (84790),RRM1 (6240),PIN1 (5300),CA9 (768),MAPK15 (225689),CCR1 (1230),AGTR2 (186),FABP1 (2168),DECR1 (1666),RIPK3 (11035),ANPEP (290),NPY1R (4886),PSMB5 (5693),GPR52 (9293),GPR148 (344561),ATIC (471),FGFR3 (2261),POLA1 (5422),TBXA2R (6915),CA6 (765),SERPINA7 (6906),ADORA1 (134),MUSK (4593),EPHB4 (2050),GC (2638),HTR2B (3357),MAPK11 (5600),SLC6A3 (6531),RARG (5916),RXFP3 (51289),DECR2 (26063),CYP3A4 (1576),HTR3E (285242),ADRA1A (148),CSNK1E (1454),GPR19 (2842),MATK (4145),NTSR2 (23620),SLC6A4 (6532),MAP3K20 (51776),STK10 (6793),SIK2 (23235),RARA (5914),NPC1L1 (29881),TRIB2 (28951),PDE2A (5138),KCNH2 (3757),SSTR3 (6753),OPRD1 (4985),FABP2 (2169),DRD1 (1812),FABP9 (646480),MCHR1 (2847),LRRK2 (120892),RDH14 (57665),ACVR2A (92),ADRB2 (154),SLC12A3 (6559),PLK4 (10733),CCR5 (1234),TNF (7124),BTK (695),APLNR (187),DRD5 (1816),HTR1A (3350),ADRA1D (146),PTGER2 (5732),EPHA2 (1969),CHRNB1 (1140),CMPK1 (51727),CCR2 (729230),IDO1 (3620),EPHA5 (2044),EGFR (1956),GPR83 (10888),DHRS4L2 (317749),CTNNB1 (1499),NR6A1 (2649),DHRS2 (10202),CHRND (1144),IRAK1 (3654),AURKB (9212),RDH11 (51109),TRHR (7201),PDE5A (8654),EPHA8 (2046),GPR37 (2861),UPP2 (151531),MERTK (10461),PGR (5241),DHRS3 (9249),NEK11 (79858),PXDNL (137902),CRABP1 (1381),BRS3 (680),PTK6 (5753),RBP7 (116362),MMP8 (4317),NFKB1 (4790),TUBB2B (347733),TXK (7294),ESRRB (2103),PDE4B (5142),HPRT1 (3251),BRAF (673),EPHA4 (2043),PIM1 (5292),LPO (4025),TUBA3C (7278),STAT5B (6777),HUNK (30811),SLCO2B1 (11309),HCK (3055),PMP2 (5375),SRMS (6725),GPR3 (2827),SLC6A16 (28968),THRA (7067),CSK (1445),TESK1 (7016),PDSS1 (23590),TSSK6 (83983),STK35 (140901),FGFR2 (2263),PTGS1 (5742),PDSS2 (57107),PKDCC (91461),PDGFRL (5157),RRH (10692),ABL2 (27),QRFPR (84109),SCN2A (6326),TPO (7173),MC3R (4159),CBR1 (873),TYMS (7298),TEC (7006),NPY4R (5540),CSNK1A1 (1452),BDH2 (56898),AGTR1 (185),CYP2C19 (1557),TNNI3K (51086),GPR22 (2845),SLC6A19 (340024),PDE6B (5158),ADORA3 (140),CXCR4 (7852),MYT1 (4661),TUBA1B (10

In [29]:
# store the cell line information (based on numpy)
# the order in cell line information is already consistent with cellline2relid_dict
np_CCLE_expression_depmap_reduced = []
for key in cellline2relid_dict.keys():
    # retrieve by cellline2relid_dict keys
    temp = np.array(CCLE_expression_depmap_reduced[CCLE_expression_depmap_reduced['cellline_name']==key])
    np_CCLE_expression_depmap_reduced.append(temp)
    
# *** but currently it does not experience 0-1 normalization ***
np_CCLE_expression_depmap_reduced = np.concatenate(np_CCLE_expression_depmap_reduced)[:, 2:]

# save the gene expression data for each cell line, still follow the order of cellline2relid_dict
np.save(prefix + midfix + 'cellline_expression.npy', np_CCLE_expression_depmap_reduced)

In [ ]:
print('np_CCLE_expression_depmap_reduced.shape:', np_CCLE_expression_depmap_reduced.shape) # final gene number for each cell line
CCLE_expression_depmap_reduced[CCLE_expression_depmap_reduced['cellline_name']=='786-0']

# obtain and further process the drug molecular graph related information for ablation experiments
drugcomb_alldruginfo_dict_lower={}
for key in drugcomb_alldruginfo_dict.keys():
    drugcomb_alldruginfo_dict_lower[key.lower()]=drugcomb_alldruginfo_dict[key]   

In [53]:
import dgl
from dgl.data import DGLDataset
import torch
import os


relid2drug_dict = {value: key for key, value in drug2relid_dict.items()}
drug_graph_edges = []
drug_graph_properties = []
# store node feature
drug_graph_nodes = []
drugid2morgan = {}
for drugid in relid2drug_dict.keys():
    # drugid: current drug id, drugcomb_alldruginfo_dict_lower: collected composite data
    druginfo = drugcomb_alldruginfo_dict_lower[relid2drug_dict[drugid]]
    drugid2morgan[drugid] = druginfo['morgan_bit']
    counter = -1
    # row in adjacent matrix
    for row in druginfo['adjacent_matrix']:
        counter += 1
        # obtain edges from current adjacent matrix
        for dst in row.nonzero()[0]:
            drug_graph_edges.append([drugid, counter, dst])

    # add self-loop
    temp = druginfo['adjacent_matrix'].shape[0]
    self_loop = [[drugid, i, i] for i in range(temp)]
    drug_graph_edges.extend(self_loop)
    # create label (graph_id) for current drug
    drug_graph_properties.append([drugid, drugid, temp])

    # store the atom numbers
    for atom in druginfo['atom_num']:
        drug_graph_nodes.append([drugid, atom])

drug_graph_edges = pd.DataFrame(drug_graph_edges, columns=['graph_id', 'src', 'dst'])

drug_graph_properties = pd.DataFrame(drug_graph_properties, columns=['graph_id', 'label', 'num_nodes'])

drug_graph_nodes = pd.DataFrame(drug_graph_nodes, columns=['graph_id', 'atom_num'])

In [54]:
drug_graph_properties.head() # label and atom nodes for each molecular graph

,graph_id,label,num_nodes
0,0,0,9
1,1,1,11
2,2,2,90
3,3,3,16
4,4,4,19


In [69]:
# count the atom number for each drug molecule
atom_set=list(set(drug_graph_nodes['atom_num']))
atom_set.sort()
atomnum2id={atom: i for i,atom in enumerate(atom_set)}
print(atomnum2id)

# test for embedding atom numbers
atom_embedding=torch.nn.Embedding(len(atom_set),64)
atom_rel_list=[atomnum2id[i] for i in atom_set]
print(atom_rel_list)

atom_embedding=atom_embedding(torch.LongTensor(atom_rel_list))
print(atom_embedding.shape)
print(torch.LongTensor(atom_rel_list).size())

print('set(drug_graph_nodes[atom_num]):', set(drug_graph_nodes['atom_num']))

# store drug related domain knowledge
with open(prefix + midfix + 'atomnum2id_dict.pickle','wb') as out_file:
    pickle.dump(atomnum2id,out_file)

# mapping from relative drug ids to correspongding ECFP6
with open(prefix + midfix + 'drugid2morgan.pickle','wb') as out_file:
    pickle.dump(drugid2morgan, out_file)

# save the molecular graph information related to involved drugs into the original data folder 
drug_graph_edges.to_csv(prefix + midfix + 'drug_graph_edges.csv',index=0)
drug_graph_properties.to_csv(prefix + midfix + 'drug_graph_properties.csv',index=0)
drug_graph_nodes.to_csv(prefix + midfix + 'drug_graph_nodes.csv',index=0)

{5: 0, 6: 1, 7: 2, 8: 3, 9: 4, 15: 5, 16: 6, 17: 7, 35: 8}
[0, 1, 2, 3, 4, 5, 6, 7, 8]
torch.Size([9, 64])
torch.Size([9])
set(drug_graph_nodes[atom_num]): {35, 5, 6, 7, 8, 9, 15, 16, 17}


In [48]:
# another test for generate drug molecular files
class DrugStrucDataset(DGLDataset):
    def __init__(self):
        super().__init__(name='drugstruc')

    def process(self):
        # read the molecular graph information related to involved drugs from the original data folder 
        edges = pd.read_csv(prefix + midfix + 'drug_graph_edges.csv')
        properties = pd.read_csv(prefix + midfix + 'drug_graph_properties.csv')
        nodes = pd.read_csv(prefix + midfix + 'drug_graph_nodes.csv')
        self.graphs = []
        self.labels = []

        # Create a graph for each graph ID from the edges table.
        # First process the properties table into two dictionaries with graph IDs as keys.
        # The label and number of nodes are values.
        label_dict = {}
        num_nodes_dict = {}
        for _, row in properties.iterrows():
            label_dict[row['graph_id']] = row['label']
            num_nodes_dict[row['graph_id']] = row['num_nodes']

        # For the edges, first group the table by graph IDs.
        edges_group = edges.groupby('graph_id')
        nodes_group = nodes.groupby('graph_id')

        # For each graph ID...
        for graph_id in edges_group.groups:
            # Find the edges as well as the number of nodes and its label.
            edges_of_id = edges_group.get_group(graph_id)
            nodes_of_id = nodes_group.get_group(graph_id)
            src = edges_of_id['src'].to_numpy()
            dst = edges_of_id['dst'].to_numpy()
            num_nodes = num_nodes_dict[graph_id]
            label = label_dict[graph_id]
            atom_features=torch.from_numpy(nodes_of_id['atom_num'].to_numpy())
            atom_features = atom_features.type(torch.float32)

            # Create a graph and add it to the list of graphs and labels.
            # in other words, create the dgl graph correspondinf to each drug molecular sub-graph and add node features for each graph here
            g = dgl.graph((src, dst), num_nodes=num_nodes)
            g.ndata['atom_num'] = atom_features
            self.graphs.append(g)
            self.labels.append(label) # each graph has a label created by its graph id

        # Convert the label list to tensor for saving.
        self.labels = torch.LongTensor(self.labels)

    def __getitem__(self, i):
        return self.graphs[i], self.labels[i]

    def __len__(self):
        return len(self.graphs)
    
# test
dataset = DrugStrucDataset()
graph, label = dataset[10]
print(graph, label) # dgl graph object for the 11th molecular graph + 11th numerical label
print('dataset.labels:', dataset.labels) # all numerical labels for every molecular graph

Graph(num_nodes=13, num_edges=39,
      ndata_schemes={'atom_num': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={}) tensor(10)
dataset.labels: tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105])


In [73]:
print('len(drug2relid_dict):', len(drug2relid_dict))
print('len(target2relid_dict):', len(target2relid_dict))
print('len(cellline2relid_dict):', len(cellline2relid_dict))

len(drug2relid_dict): 106
len(target2relid_dict): 12217
len(cellline2relid_dict): 60
